# Week 8 Homework

Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [3]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [4]:

import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

## Exercise 11-1
Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [6]:
# Getting the birth data that are more than 30 weeks since the bets are placed during the 30th week
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [7]:
# Checking the columns
live.columns.nunique()

244

In [8]:
# Join the data form the respondent table
import nsfg
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3331)

In [9]:
# Data mining for variables that have prediction power
def GoMining(df, top=30):
    """Searches for variables that predict pregnancey length.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs sorted by rsquared in descending order, top 30 results by default
    """
    import patsy
    import statsmodels.formula.api as smf
    import re
    variables = []
    for name in df.columns:
        if name != 'prglngth':
            try:
                if df[name].var() < 1e-7:
                    continue
    
                formula = 'prglngth ~ ' + name
                model = smf.ols(formula, data=df)
                if model.nobs < len(df)/2:
                    continue
    
                results = model.fit()
            except (ValueError, TypeError, patsy.PatsyError) as e:
                continue
            
            variables.append((results.rsquared, name))
    
    variables = sorted(variables, key = lambda x: x[0], reverse = True)

    # Reads Stata dictionary files for NSFG data and maps variables names to descriptions
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = pd.concat([vars1, vars2])
    all_vars.index = all_vars.name

    for r2, name in variables[:top]:
        # remove the _r suffix if any variable name has it
        key = re.sub('_r$', '', name) 
        try:
            # search the description of the variable name
            desc = all_vars.loc[key].desc
        except (KeyError, IndexError):
            print(name, r2)
        else:
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)

In [10]:
GoMining(join)

wksgest 0.8062434116139245 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120214
birthwgt_lb 0.11977307804917159 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.1037254220458328 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592757 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022053775796467057 PRGLNGTH IMPUTATION FLAG
canhaver 0.006050495268196787 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
datcon01_i 0.005817755299879157 DATCON01 IMPUTATION FLAG
con1mar1_i 0.00554637613624176 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.004577565785536919 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
mar1con1_i 0.0031508022538629943 MAR1CON1 IMPUTATION FLAG
anynurse 0.002452024883710213 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.002369183944665898 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.0022493894337988207 BC-1 HOW PREGNANCY ENDED - 1ST MENTION
marout11_i 0.0022436279681069538 MAROUT11 IMPUTATI

According to the top 30 variables in terms of highest R^2, Curren livng quarters owned/rented, etc looks like a single variable that makes the best prediction and is known before the birth.

## Exercise 11-3 
If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [13]:
# Check the variable
join['numbabes'].value_counts()

numbabes
2     3105
3     2363
1     1479
4     1152
5      432
6      164
7       93
8       54
10      18
9       12
16       7
22       5
Name: count, dtype: int64

In [14]:
# Look for any age variables
vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

all_vars = pd.concat([vars1, vars2])
all_vars.index = all_vars.name

pd.set_option('display.max_rows', None)

all_vars[['name','desc']][all_vars.desc.str.contains(r'\bage\b', case=False, na=False)]

,name,desc
name,,
ageatend,ageatend,BC-4B R'S AGE AT PREGNANCY'S END DATE
hpageend,hpageend,BC-4C FATHER'S AGE AT PREGNANCY'S END DATE
kidage,kidage,CURRENT AGE (IN MOS) OF R'S CHILD(REN) FROM TH...
hpagelb,hpagelb,BD-6 FATHER'S AGE AT TIME OF CHILD(REN) S BIRTH
lastage,lastage,AGE (IN MOS) WHEN CHILD LAST LIVED W/R-1ST FRO...
frsteatd_n,frsteatd_n,BH-3 AGE (MOS/WKS/DAY) WHEN 1ST SUPPLEMENTED -...
frsteatd,frsteatd,AGE (IN MOS) WHEN 1ST SUPPLEMENTED - 1ST FROM ...
ageqtnur_n,ageqtnur_n,BH-5 AGE (MOS/WKS/DAY) WHEN STOPPED BREASTFEED...
ageqtnur,ageqtnur,AGE (IN MOS) WHEN R'STOPPED NURSING CHILD - 1S...


age_r looks like the variable that's fitting the question.

In [16]:
# Look for any college variables
pd.options.display.max_colwidth = 200
all_vars[all_vars.desc.str.contains(r'\bcollege\b', case=False, na=False)]

,start,type,name,fstring,desc,end
name,,,,,,
havedeg,52,<class 'int'>,havedeg,%1f,AF-10 WHETHER R HAS ANY COLLEGE OR UNIVERSITY DEGREES,53.0
degrees,53,<class 'int'>,degrees,%1f,AF-11 HIGHEST COLLEGE OR UNIVERSITY DEGREE,54.0


In [17]:
# Check the havedeg
join.havedeg.unique()

array([ 1., nan,  5.])

Looks like as long the value is not null, the respondent has a college degree

In [19]:
# Check the data distribution to confirm the guess above
join.havedeg.value_counts(dropna=False)

havedeg
NaN    4645
1.0    2137
5.0    2102
Name: count, dtype: int64

In [25]:
# Train the model
import statsmodels.formula.api as smf

formula = ('numbabes ~ age_r + C(race) + havedeg + totincr')
results = smf.poisson(formula, data=join).fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.590692
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 4239
Model:                        Poisson   Df Residuals:                     4233
Method:                           MLE   Df Model:                            5
Date:                Sat, 27 Jul 2024   Pseudo R-squ.:                 0.01338
Time:                        12:24:33   Log-Likelihood:                -6742.9
converged:                       True   LL-Null:                       -6834.4
Covariance Type:            nonrobust   LLR p-value:                 1.292e-37
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.3549      0.069      5.171      0.000       0.220       0.489
C(race)[T.2]    -0.0559      0.024     -2.300      0.021      -0.103      -0.008
C(race)[T.3]    -0.0726      0.041     -1.769      0.077      -0.153       0.008
age_r            0.0207      0.002     12.018      0.000       0.017       0.024
havedeg          0.0125      0.005      2.368      0.018       0.002       0.023
totincr         -0.0198      0.003     -6.807      0.000      -0.025      -0.014
================================================================================
"""

According to the text book, totincr increaments at \\$5000, so \\$75000 will be level 15. 
<br>
For race, 1 will be black.
<br>
Therefore, we can make the prediciton based on the following:

In [28]:
columns = ['age_r', 'race', 'havedeg','totincr']
new = pd.DataFrame([[35,1,1,15]], columns=columns)
results.predict(new)

0    2.216382
dtype: float64

We can predict she has born about 2 to 3 children.

## Exercise 11-4 
If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [31]:
# Check the response variable
live.rmarital.value_counts(dropna=False)

rmarital
1    5027
6    1403
2     914
4     860
5     575
3     105
Name: count, dtype: int64

I will assume that 1,2,3,4,5,6 represents: married, cohabitating, widowed, divorced, separated, or never married

In [33]:
# Check the high school graduate variable
all_vars[all_vars.desc.str.contains(r'\bhigh school\b', case=False, na=False)]

,start,type,name,fstring,desc,end
name,,,,,,
havedip,46,<class 'int'>,havedip,%1f,AF-5 WHETHER R HAS HIGH SCHOOL DIPLOMA OR GED OR BOTH,47.0
dipged,47,<class 'int'>,dipged,%1f,AF-6 WHICH ONE R HAS: HIGH SCHOOL DIPLOMA OR GED OR BOTH,48.0
cmhsgrad,48,<class 'int'>,cmhsgrad,%4f,CENTURY MONTH OF HIGH SCHOOL GRADUATION,52.0


In [34]:
# Check the havedip variable
join.havedip.value_counts(dropna=False)

havedip
1.0    6798
5.0    1946
NaN     137
9.0       3
Name: count, dtype: int64

I will assume 1 means high school 5 means GED, and 9 means both.
<br>
As for income, since she makes \\$45000, and the income level increments at \\$5000, so she's at level 9.

In [36]:
# Build the model
formula = ('rmarital ~ age_r + C(race) + havedip + totincr')
results = smf.mnlogit(formula, data=join).fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.088245
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8747
Model:                        MNLogit   Df Residuals:                     8717
Method:                           MLE   Df Model:                           25
Date:                Sat, 27 Jul 2024   Pseudo R-squ.:                  0.1636
Time:                        12:25:00   Log-Likelihood:                -9518.9
converged:                       True   LL-Null:                       -11380.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.0081      0.227      8.844      0.000       1.563       2.453
C(race)[T.2]    -0.9026      0.089    -10.181      0.000      -1.076      -0.729
C(race)[T.3]    -0.6709      0.136     -4.925      0.000      -0.938      -0.404
age_r           -0.0612      0.006    -10.262      0.000      -0.073      -0.050
havedip          0.1528      0.022      7.103      0.000       0.111       0.195
totincr         -0.1558      0.011    -13.752      0.000      -0.178      -0.134
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -5.4312      0.782     -6.941      0.000      -6.965      -3.898
C(race)[T.2]    -0.4743      0.238     -1.993      0.046      -0.941      -0.008
C(race)[T.3]     0.0199      0.334      0.060      0.952      -0.635       0.675
age_r            0.1234      0.019      6.451      0.000       0.086       0.161
havedip          0.0248      0.056      0.442      0.659      -0.085       0.135
totincr         -0.3388      0.031    -10.852      0.000      -0.400      -0.278
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.5967      0.264     -6.050      0.000      -2.114      -1.079
C(race)[T.2]    -0.3149      0.094     -3.342      0.001      -0.500      -0.130
C(race)[T.3]    -0.7261      0.172     -4.230      0.000      -1.063      -0.390
age_r            0.0678      0.007     10.213      0.000       0.055       0.081
havedip         -0.1713      0.027     -6.245      0.000      -0.225      -0.118
totincr         -0.2313      0.011    -20.738      0.000      -0.253      -0.209
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.6905      0.281      2.458      0.014       0.140       1.241
C(race)[T.2]    -1.0511      0.104    -10.105      0.000      -1.255      -0.847
C(race)[T.3]    -0.5959      0.156     -3.816      0.000      -0.902      -0.290
age_r            0.0053      0.007      0.720      0.471      -0.009       0.020
havedip          0.0222      0.026      0.842      0.400      -0.029       0.074
totincr         -0.2954      0.014    -20.772      0.000      -0.323      -0.268
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        5.4181      0.219     24.763      0.000       4.989       5.847
C

In [37]:
columns = ['age_r', 'race', 'havedip','totincr']
new = pd.DataFrame([[25,2,1,9]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.674202,0.126373,0.001953,0.057003,0.03839,0.10208


According to our assumotion of what each unique value in the repsonse variable means, the probability of married, cohabitating, widowed, divorced, separated, or never married is shown above, respectively. Therefore she's most likely married.